In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [39]:
from src.inference import get_feature_store

In [40]:
print(ts_data.dtypes)  # Check the data types of all columns


pickup_hour           datetime64[us, Etc/UTC]
pickup_location_id                      int32
rides                                   int32
dtype: object


In [41]:
print(type(fetch_data_from), fetch_data_from)
print(type(fetch_data_to), fetch_data_to)


<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2025-02-03 21:40:34.617621+00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2025-03-04 20:40:34.617621+00:00


In [42]:
print(ts_data['pickup_hour'].isna().sum())  # Count NaT (missing values)


TypeError: int() argument must be a string, a bytes-like object or a real number, not '_NoValueType'

In [43]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime in UTC  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# Define time range
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=29)

# Debugging: Ensure timestamps are correct
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")

# Get Feature View
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

# Fetch Data
ts_data = feature_view.get_batch_data(
    start_time=fetch_data_from - timedelta(days=1),
    end_time=fetch_data_to + timedelta(days=1),
)

# Ensure pickup_hour is datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'])


2025-03-04 16:45:24,408 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 16:45:24,412 INFO: Initializing external client
2025-03-04 16:45:24,413 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 16:45:25,067 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703
Fetching data from 2025-02-03 21:45:24.408236+00:00 to 2025-03-04 20:45:24.408236+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.27s) 


In [ ]:
ts_data

In [44]:
# Apply filter correctly
ts_data = ts_data[(ts_data['pickup_hour'] >= fetch_data_from) & (ts_data['pickup_hour'] <= fetch_data_to)]

print(ts_data.head())  # Check the filtered data

TypeError: int() argument must be a string, a bytes-like object or a real number, not '_NoValueType'

In [31]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 16:37:47,736 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 16:37:47,741 INFO: Initializing external client
2025-03-04 16:37:47,741 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 16:37:48,279 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703
Fetching data from 2025-02-03 21:37:47.736757+00:00 to 2025-03-04 20:37:47.736757+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.56s) 


TypeError: int() argument must be a string, a bytes-like object or a real number, not '_NoValueType'

In [ ]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-03-01 15:00:00+00:00,216,3
1,2025-02-20 16:00:00+00:00,217,2
2,2025-02-11 06:00:00+00:00,255,0
3,2025-02-12 09:00:00+00:00,21,0
4,2025-02-18 00:00:00+00:00,255,2
...,...,...,...
174942,2025-02-12 01:00:00+00:00,69,0
174943,2025-02-16 22:00:00+00:00,202,0
174944,2025-02-18 17:00:00+00:00,113,80
174945,2025-02-26 15:00:00+00:00,245,0


In [ ]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 21:00:00+00:00,2,0
1,2025-02-03 22:00:00+00:00,2,0
2,2025-02-03 23:00:00+00:00,2,0
3,2025-02-04 00:00:00+00:00,2,0
4,2025-02-04 01:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 15:00:00+00:00,263,126
174441,2025-03-04 16:00:00+00:00,263,113
174442,2025-03-04 17:00:00+00:00,263,117
174443,2025-03-04 18:00:00+00:00,263,132


In [ ]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 174946
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [ ]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [ ]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 174946
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [ ]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [ ]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,3,0,1,0,2,1,0,1,0,4,...,0,0,0,1,1,3,2,0,216,2025-03-03 02:00:00
1,2,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,217,2025-02-04 14:00:00
2,0,2,1,0,24,0,0,0,3,1,...,0,1,1,1,2,1,1,2,255,2025-03-03 03:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,21,2025-02-24 04:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,171,2025-02-19 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,208,2025-02-18 13:00:00
247,42,70,5,145,139,86,5,21,43,25,...,46,4,465,5,56,6,93,110,114,2025-02-21 05:00:00
248,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,123,2025-02-04 00:00:00
249,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,1,0,0,202,2025-02-04 16:00:00


In [ ]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 15:38:39,402 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 15:38:39,407 INFO: Initializing external client
2025-03-04 15:38:39,407 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 15:38:39,968 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703
Fetching data from 2025-02-03 20:38:39.402583+00:00 to 2025-03-04 19:38:39.402583+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.05s) 


In [ ]:
current_date

Timestamp('2025-03-04 20:38:39.402583+0000', tz='Etc/UTC')

In [ ]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 21:00:00
1,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,3,2025-03-03 21:00:00
2,3,6,2,3,6,2,0,1,1,2,...,2,2,3,0,1,1,0,0,4,2025-03-03 21:00:00
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 21:00:00
4,2,0,0,0,0,1,0,0,1,2,...,3,4,1,3,4,1,5,2,7,2025-03-03 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,259,2025-03-03 21:00:00
247,1,0,0,0,0,0,0,0,0,1,...,3,2,5,1,6,1,1,0,260,2025-03-03 21:00:00
248,16,11,3,1,3,0,0,2,2,0,...,36,32,26,33,43,45,24,22,261,2025-03-03 21:00:00
249,22,4,6,1,3,1,1,3,19,63,...,71,82,97,87,82,103,65,37,262,2025-03-03 21:00:00


In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 15:39:11,209 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 15:39:11,228 INFO: Initializing external client
2025-03-04 15:39:11,228 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 15:39:11,791 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703


In [ ]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [ ]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,1.0
...,...,...
246,259,0.0
247,260,0.0
248,261,14.0
249,262,21.0


In [ ]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 161, 237, 138, 186, 162, 163, 142, 230,  68])